In [17]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv


In [18]:
load_dotenv()

True

In [ ]:
import os
os.environ.pop("GROQ_API_KEY", None)
print("GROQ:", os.getenv("GROQ_API_KEY"))

GROQ: None


In [20]:
os.environ.pop("GROQ_API_KEY", None)

In [41]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="text-generation"
)


In [42]:
llm = ChatHuggingFace(llm=llm)

In [43]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [44]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = llm.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [45]:
# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [50]:
# Execute
initial_state = {
    'question': 'Who won 2024 T20 WC and by how many runs? Who was Player of match? And give summary about final!!'
}
final_state = workflow.invoke(intial_state)

print(final_state['answer'])

**2024 ICC Men’s T20 World Cup – Final Recap**

| Item | Details |
|------|---------|
| **Winner** | **India** |
| **Runner‑up** | South Africa |
| **Result** | India won by **7 runs** |
| **Player of the Match** | **Virat Kohli** (India) – 76 runs off 49 balls |
| **Venue** | **Jersey City, New Jersey – Red Bull Arena** (the first ICC World Cup final ever played in the United States) |
| **Date** | **29 June 2024** |

---

### How the Match Unfolded

| innings | Score | Key moments |
|---------|-------|-------------|
| **India (batting first)** | **176/7 (20 overs)** | • Rohit Sharma gave a quick start (31 off 20).<br>• Virat Kohli took charge at 86/2, smashing 76 off 49 with 8 fours and 3 sixes, anchoring the chase.<br>• Hardik Pandya added a handy 38* off 22 to push the total past the 170 mark.<br>• Jasprit Bumrah bowled the death overs, finishing with 2/28. |
| **South Africa (chasing)** | **169/8 (20 overs)** | • Quinton de Kock’s brisk 44 set a good platform.<br>• Aamer Salah (55